In [ ]:
import geopandas as gpd
import pandas as pd

In [ ]:
gdf_mesh = gpd.read_file('1km_mesh_suikei_2018_shape_27.zip!1km_mesh_2018_27.shp').loc[:, ["MESH_ID", "SHICODE", "geometry"]]
gdf_mesh

In [ ]:
# http://cellmap.rukihena.com/mls44011.json
df_MLS_1 = pd.read_json('mls44011.json').fillna('')
df_MLS_1

In [ ]:
# ECI(cell)範囲指定
# https://mls.js2hgw.com/wiki/?用語集#a19d1c53

# PCI(unit)範囲指定しマクロセルを抽出
# https://mls.js2hgw.com/wiki/?用語集#k2c8cab6

df_MLS_2 = df_MLS_1.query('(226492172 >= cell >= 4194305) & (389 >= unit >= 0) & (area < 49154)').copy()
df_MLS_2

In [ ]:
# 特殊TAC + 都道府県別TAC (移動局・Casaは除く)
TAC = [111, 112, 113, 114, 211, 212, 213, 214, 311, 312, 313, 314, 1025, 1026, 1027, 1028, 1029, 2049, 2050, 3073, 3074, 4097, 4098, 5121, 5122, 6145, 6146, 7169, 7170, 7171, 8193, 8194, 8195, 8196, 9217, 9218, 10241, 10242, 11265, 11266, 11267, 11268, 12289, 12290, 12291, 12292, 13313, 13314, 13315, 13316, 15361, 15362, 15363, 15364, 15365, 15366, 16385, 16386, 16387, 17409, 18433, 19457, 20481, 21505, 21506, 21507, 22529, 22530, 22531, 23553, 23554, 23555, 23556, 24577, 24578, 24579, 24580, 25511, 25512, 25513, 25514, 25601, 25602, 26625, 26626, 27649, 27650, 28673, 28674, 28675, 29697, 29698, 29699, 29700, 30721, 30722, 31745, 31746, 32769, 33411, 33793, 33794, 34817, 34818, 34819, 35841, 35842, 35843, 36865, 36866, 37889, 38913, 39937, 39938, 40961, 40962, 41985, 41986, 41987, 41988, 41989, 41990, 43009, 43010, 44033, 44034, 45057, 45058, 45059, 46081, 46082, 47105, 47106, 48129, 48130, 49153, 49154]

df_MLS_3 = df_MLS_2[df_MLS_2['area'].isin(TAC)]
df_MLS_3

In [ ]:
# 緯度経度をgeometryに変換
gdf_MLS = gpd.GeoDataFrame(df_MLS_3, geometry = gpd.points_from_xy(df_MLS_3.lon, df_MLS_3.lat), crs="EPSG:6668")
gdf_MLS

In [ ]:
# print(gdf_mesh.crs) epsg:4612
# print(gdf_MLS.crs) EPSG:6668

gdf_mesh = gdf_mesh.to_crs(epsg=6668)

In [ ]:
mergedGdf = gpd.sjoin(gdf_mesh, gdf_MLS[['area', 'samples', 'geometry']])
mergedGdf = mergedGdf[['MESH_ID', 'SHICODE', 'area', 'samples', 'geometry']].copy()
mergedGdf.sort_values('MESH_ID', inplace=True)
mergedGdf.reset_index(drop=True, inplace=True)
mergedGdf

In [ ]:
df0 = pd.DataFrame(mergedGdf)
df1 = df0.loc[df0.groupby('MESH_ID')['samples'].idxmax()]
df1.sort_values('MESH_ID', inplace=True)
df1.reset_index(drop=True, inplace=True)
df1

In [ ]:
gdf_TAC = gpd.GeoDataFrame(df1[['area', 'geometry']], crs='EPSG:6668')
gdf_TAC = gdf_TAC.dissolve(by = 'area', as_index = False)
gdf_TAC

In [ ]:
import folium
import branca.colormap as cm

In [ ]:
colormap = cm.StepColormap(
    colors=['#FF0000', '#FF9932', '#FFFF00', '#65FF32', '#008000', '#32CCFF', '#3265FF', '#FF32FF', '#ffffff', '#656565'],
    vmin=0,
    vmax=10,
    index=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
)

def ReturnNumber(nums):
    str_nums = str(nums)
    return int(str(int(str_nums[-1]) + int(str_nums[-2]))[-1])

In [ ]:
# ベースマップ
map = folium.Map(
    tiles = None,
    location = [34.5496419, 135.7788521],
    zoom_start = 9,
    control_scale = True
)

# 国土地理院(白地図)
folium.raster_layers.TileLayer(
    'https://cyberjapandata.gsi.go.jp/xyz/blank/{z}/{x}/{y}.png',
    name = "国土地理院(白地図)",
    attr = "<a href='https://maps.gsi.go.jp/development/ichiran.html'>国土地理院</a>"
).add_to(map)

# Googleマップ標準
folium.raster_layers.TileLayer(
    'https://{s}.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
    subdomains = ['mt0','mt1','mt2','mt3'],
    name = "Google Map",
    attr = "<a href='https://developers.google.com/maps/documentation' target='_blank'>Google Map</a>"
).add_to(map)

In [ ]:
folium.GeoJson(
    data = gdf_TAC.to_json(),
    style_function = lambda x:{
        "fillColor": colormap(ReturnNumber(x['properties']['area'])),
        'fillOpacity': 0.7,
        "stroke": False
    },
    name = 'TAC',
    show = True,
    tooltip = folium.features.GeoJsonTooltip(
        fields=['area', ],
        aliases=['TAC', ]
    )
).add_to(map)

In [ ]:
# レイヤーコントロール
folium.LayerControl().add_to(map)

In [ ]:
map